# E-04.  lyrics_generator

## 1. 데이터 읽어오기

In [1]:
import glob
import os, re
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

In [2]:

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)
print(f'The number of lyrics : {len(txt_list)}')


raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("The number of raw corpus:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

The number of lyrics : 49
The number of raw corpus: 187088
Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me']


---

## 2. 데이터 정제
- `preprocess_sentence()`함수 만들기
-  토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외할 것
(다른 데이터들이 과도한 Padding을 갖게 하므로)

**정규 표현식을 이용해서 필터링**
1. 소문자로 바꾸고, 양쪽 공백을 지우기
2. 특수문자 양쪽에 공백을 넣기
3. 여러개의 공백은 하나의 공백으로 바꾸기
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾸기
5. `[]` 해당하는 부분은 파트 분배 임으로 필터링
6. `()` 해당하는 부분은 코러스 부분, 연속된 부분을 표기한 것이므로 필터링
7. 양쪽 공백 지우기
8. 문장 시작에는 `<start>`, 끝에는 `<end>`를 추가
    

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = re.sub(r"(\[.*\])", '', sentence) # 5
    sentence = re.sub(r"(\(.*\))", '', sentence) # 6
    sentence = sentence.strip() # 7
    sentence = '<start> ' + sentence + ' <end>' # 8
    return sentence


In [5]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == "]": continue    
    if sentence[-1] == ")": continue     
    if len(sentence) == 0 or len(sentence.split()) > 15: continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]


['<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> somethin ain t right when we talkin <end>',
 '<start> somethin ain t right when we talkin <end>',
 '<start> look like you hidin your problems <end>']

----

## 3. 평가 데이터셋 분리
- `tokenize()` 함수로 데이터를 `Tensor`로 변환
-  `sklearn` 모듈의 `train_test_split()` 함수를 사용해 훈련 데이터와 평가 데이터를 분리
- 단어장의 크기는 12,000 이상 으로 설정
- 총 데이터의 20% 를 평가 데이터셋으로 사용

In [6]:
def tokenize(corpus):
    # 15000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    # 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    5   90 ...   10   12    3]
 [   2   41  132 ...    0    0    0]
 [   2    5   38 ...    0    0    0]
 ...
 [   2   86  713 ...    0    0    0]
 [   2  209    3 ...    0    0    0]
 [   2    9 1376 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f578804f5d0>


In [10]:
# 단어사전이 어떻게 구축되었는지 확인한다.
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


- 텐서를 소스와 타겟으로 분리

In [7]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2   5  90 102  59  31 161   4  11 132  24  29  10  12]
[  5  90 102  59  31 161   4  11 132  24  29  10  12   3]


 - train, validation 데이터 분리

In [8]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                       tgt_input,
                                       test_size=0.2,
                                       random_state=7)



print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (127837, 14)
Target Train: (127837, 14)


In [9]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 7000개와, 
#여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다 
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

 - 데이터셋 객체 생성
     
     - 텐서로 생성된 데이터를 이용해 tf.data.Dataset객체를 생성
     - tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성

----
## 4. 인공지능 만들기

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, dropout = 0.3, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 15001), dtype=float32, numpy=
array([[[ 1.86683275e-04, -2.55409715e-04,  1.00842029e-04, ...,
          7.69045728e-05,  1.83778378e-04,  2.93905003e-04],
        [-3.01008622e-05, -4.27234889e-04,  9.60441776e-06, ...,
          1.14403156e-04,  4.37825569e-04,  6.93772337e-04],
        [ 4.30815184e-04, -5.61389374e-04,  5.99502127e-05, ...,
         -1.62298660e-04,  1.09001226e-03,  1.42291770e-03],
        ...,
        [-5.55209772e-05,  5.37578831e-04,  2.20520218e-04, ...,
         -6.09352079e-04, -1.06444478e-03,  1.26104581e-03],
        [ 9.00101586e-05,  6.16391073e-04, -5.45953284e-04, ...,
         -5.19308844e-04, -9.02565545e-04,  1.18848670e-03],
        [ 5.36918931e-04,  8.72895063e-04, -1.43505679e-03, ...,
         -4.22512385e-04, -5.62763482e-04,  1.06089213e-03]],

       [[ 1.86683275e-04, -2.55409715e-04,  1.00842029e-04, ...,
          7.69045728e-05,  1.83778378e-04,  2.93905003e-04],
        [-3.01008622e-05, -4.27234889e-04,  9

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  15361024  
_________________________________________________________________
lstm (LSTM)                  multiple                  25174016  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  30737049  
Total params: 104,834,713
Trainable params: 104,834,713
Non-trainable params: 0
_________________________________________________________________


---
### Model 1
- 하이퍼 파라미터를 여러 조합으로 바꿔서 모델을 학습시켜봤으나, val_loss 값이 2.2 이하로 떨어지질 않았다.
- 고민하던 중 dropout을 추가해보았다.
- 아래 결과는 Rnn1과 Rnn2 LSTM 에 dropout을 각각 0.5로 준 결과이다.

In [90]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
data1 = model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val))

Epoch 1/10
520/520 [==============================] - 542s 1s/step - loss: 4.1265 - val_loss: 3.1648
Epoch 2/10
520/520 [==============================] - 544s 1s/step - loss: 3.0568 - val_loss: 2.9022
Epoch 3/10
520/520 [==============================] - 538s 1s/step - loss: 2.7243 - val_loss: 2.7235
Epoch 4/10
520/520 [==============================] - 554s 1s/step - loss: 2.4257 - val_loss: 2.6019
Epoch 5/10
520/520 [==============================] - 585s 1s/step - loss: 2.1624 - val_loss: 2.5166
Epoch 6/10
520/520 [==============================] - 537s 1s/step - loss: 1.9331 - val_loss: 2.4562
Epoch 7/10
520/520 [==============================] - 537s 1s/step - loss: 1.7484 - val_loss: 2.4163
Epoch 8/10
520/520 [==============================] - 539s 1s/step - loss: 1.5935 - val_loss: 2.3939
Epoch 9/10
520/520 [==============================] - 593s 1s/step - loss: 1.4626 - val_loss: 2.3881
Epoch 10/10
520/520 [==============================] - 545s 1s/step - loss: 1.3550 - val_lo

### Model 2
- dropout을 Rnn1, Rnn2 LSTM에 모두 0.5씩 설정하였으나, Val_loss값이 유의미하게 떨어지지 않아 다시 설정해보았다.
- 아래 학습에서는 dropout을 Rnn1 LSTM에만 0.5 주었다.

In [98]:
# dropout을 Rnn1에만 0.5로 준 것
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none')


model.compile(loss=loss, optimizer=optimizer)
data2 = model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val),  verbose=1)

Epoch 1/10
520/520 [==============================] - 543s 1s/step - loss: 3.3308 - val_loss: 3.0312
Epoch 2/10
520/520 [==============================] - 541s 1s/step - loss: 2.8821 - val_loss: 2.8033
Epoch 3/10
520/520 [==============================] - 541s 1s/step - loss: 2.5304 - val_loss: 2.6545
Epoch 4/10
520/520 [==============================] - 547s 1s/step - loss: 2.2155 - val_loss: 2.5535
Epoch 5/10
520/520 [==============================] - 595s 1s/step - loss: 1.9405 - val_loss: 2.4866
Epoch 6/10
520/520 [==============================] - 541s 1s/step - loss: 1.7112 - val_loss: 2.4443
Epoch 7/10
520/520 [==============================] - 541s 1s/step - loss: 1.5144 - val_loss: 2.4165
Epoch 8/10
520/520 [==============================] - 600s 1s/step - loss: 1.3489 - val_loss: 2.4138
Epoch 9/10
520/520 [==============================] - 534s 1s/step - loss: 1.2162 - val_loss: 2.4176
Epoch 10/10
520/520 [==============================] - 539s 1s/step - loss: 1.1117 - val_lo

### Model 3

- LSTM 인자에 dropdout 을 추가하여도 val_loss가 크게 감소할 기미를 보이지 않아, callback 함수를 이용해보았다.
- ReduceLROnPlateau: 모델의 개선이 없을 경우, Learning Rate를 조절해 모델의 개선을 유도하는 콜백함수
- monitor='val_loss': 기준이 되는 값. val_loss가 더이상 감소되지 않을 경우 ReduceLROnPlateau을 적용
- factor: Learning rate를 얼마나 감소시킬 지 정하는 인자값
- patience: Training이 진행됨에도 더이상 monitor되는 값의 개선이 없을 경우, 최적의 monitor 값을 기준으로 몇 번의 epoch을 진행하고, learning rate를 조절할 지의 값
- mode: monitor되는 값이 최소가 되어야 하는지, 최대가 되어야 하는지 알려주는 인자
- min_lr: Learning rate의 하한선


In [129]:
#dropout 0.5 Rnn1
from keras.callbacks import ReduceLROnPlateau

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none')

reduce_lr  = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                           patience=2, mode="min", min_lr=0.001)


model.compile(loss=loss, optimizer=optimizer)
data3 = model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val), verbose=1, callbacks=[reduce_lr])

Epoch 1/10
520/520 [==============================] - 596s 1s/step - loss: 3.4947 - val_loss: 3.0316
Epoch 2/10
520/520 [==============================] - 537s 1s/step - loss: 2.8890 - val_loss: 2.7933
Epoch 3/10
520/520 [==============================] - 540s 1s/step - loss: 2.5171 - val_loss: 2.6409
Epoch 4/10
520/520 [==============================] - 538s 1s/step - loss: 2.1925 - val_loss: 2.5342
Epoch 5/10
520/520 [==============================] - 590s 1s/step - loss: 1.9217 - val_loss: 2.4612
Epoch 6/10
520/520 [==============================] - 540s 1s/step - loss: 1.6708 - val_loss: 2.4172
Epoch 7/10
520/520 [==============================] - 545s 1s/step - loss: 1.4642 - val_loss: 2.3912
Epoch 8/10
520/520 [==============================] - 593s 1s/step - loss: 1.2986 - val_loss: 2.3888
Epoch 9/10
520/520 [==============================] - 539s 1s/step - loss: 1.1651 - val_loss: 2.3999
Epoch 10/10
520/520 [==============================] - 536s 1s/step - loss: 1.0692 - val_lo

### Model 4
- callback 함수를 사용했음에도 val_loss가 2.2 아래로 떨어지질 않는다.
- 혹시 dropout으로 하나의 Rnn의 뉴런만 죽인게 영향이 있을까 싶어서 dropout 값을 0.3으로 Rnn1, 2에 모두 줘봤다.

In [134]:
# dropout = 0.3 x2
from keras.callbacks import ReduceLROnPlateau

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none')

reduce_lr  = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                           patience=2, mode="min", min_lr=0.001)


model.compile(loss=loss, optimizer=optimizer)
data4 = model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val), verbose=1, callbacks=[reduce_lr])

Epoch 1/10
520/520 [==============================] - 562s 1s/step - loss: 4.1491 - val_loss: 3.1913
Epoch 2/10
520/520 [==============================] - 539s 1s/step - loss: 3.0772 - val_loss: 2.9205
Epoch 3/10
520/520 [==============================] - 539s 1s/step - loss: 2.7371 - val_loss: 2.7393
Epoch 4/10
520/520 [==============================] - 540s 1s/step - loss: 2.4324 - val_loss: 2.6060
Epoch 5/10
520/520 [==============================] - 538s 1s/step - loss: 2.1575 - val_loss: 2.5104
Epoch 6/10
520/520 [==============================] - 539s 1s/step - loss: 1.9189 - val_loss: 2.4438
Epoch 7/10
520/520 [==============================] - 548s 1s/step - loss: 1.7138 - val_loss: 2.3988
Epoch 8/10
520/520 [==============================] - 541s 1s/step - loss: 1.5328 - val_loss: 2.3748
Epoch 9/10
520/520 [==============================] - 540s 1s/step - loss: 1.3917 - val_loss: 2.3637
Epoch 10/10
520/520 [==============================] - 541s 1s/step - loss: 1.2708 - val_lo

### Model 5
- dropout을 Rnn1, Rnn2 LSTM에 모두 0.3씩 설정하였으나, Val_loss값이 유의미하게 떨어지지 않아 다시 설정해보았다.
- 아래 학습에서는 dropout을 Rnn2 LSTM에만 0.3 주었다.
- 또한 혹시 하이퍼파라미터를 변경하면 영향이 있을까 싶어 임베딩 사이즈를 1024로 높여보았다.
- 전처리의 문제일까 싶어 문장 필터를 한번 더 넣어줬다 (문장 길이 제한)

In [13]:
# dropout = 0.3 + 전처리 추가(문장 길이에 대한 옵션을 더블로 줬음)+ 임베딩 1024
from keras.callbacks import ReduceLROnPlateau

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none')

reduce_lr  = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                           patience=2, mode="min", min_lr=0.05)


model.compile(loss=loss, optimizer=optimizer)
data5 = model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val), verbose=1, callbacks=[reduce_lr])

Epoch 1/10
499/499 [==============================] - 603s 1s/step - loss: 2.9187 - val_loss: 2.7627
Epoch 3/10
499/499 [==============================] - 597s 1s/step - loss: 2.5538 - val_loss: 2.5856
Epoch 4/10
499/499 [==============================] - 597s 1s/step - loss: 2.2269 - val_loss: 2.4598
Epoch 5/10
499/499 [==============================] - 607s 1s/step - loss: 1.9422 - val_loss: 2.3722
Epoch 6/10
499/499 [==============================] - 597s 1s/step - loss: 1.7045 - val_loss: 2.3147
Epoch 7/10
499/499 [==============================] - 598s 1s/step - loss: 1.5023 - val_loss: 2.2797
Epoch 8/10
499/499 [==============================] - 599s 1s/step - loss: 1.3400 - val_loss: 2.2707
Epoch 9/10
499/499 [==============================] - 598s 1s/step - loss: 1.2125 - val_loss: 2.2734
Epoch 10/10
499/499 [==============================] - 600s 1s/step - loss: 1.1194 - val_loss: 2.2915


### Model 6
- model 5에서 dropout, callback, 임베딩 사이즈, 데이터 전처리까지 했음에도 큰 의미가 없었다....
- 마지막으로 양방향 Rnn을 사용해보았다...
- val_loss는 2.2 아래로 떨어졌으나, 오버피팅을 확인할 수 있었다.

In [24]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
        self.rnn_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_size, dropout = 0.3, return_sequences=True))
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [25]:
# dropout = 0.3 + 전처리 추가(문장 길이에 대한 옵션을 더블로 줬음)+ 임베딩 1024+ 양방향 Rnn
from keras.callbacks import ReduceLROnPlateau

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none')

reduce_lr  = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                           patience=2, mode="min", min_lr=0.05)


model.compile(loss=loss, optimizer=optimizer)
data6 = model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val), verbose=1, callbacks=[reduce_lr])

Epoch 1/10
499/499 [==============================] - 1325s 3s/step - loss: 3.1529 - val_loss: 0.6304
Epoch 2/10
499/499 [==============================] - 1278s 3s/step - loss: 0.4672 - val_loss: 0.2889
Epoch 3/10
499/499 [==============================] - 1276s 3s/step - loss: 0.1617 - val_loss: 0.1782
Epoch 4/10
499/499 [==============================] - 1274s 3s/step - loss: 0.0489 - val_loss: 0.1393
Epoch 5/10
499/499 [==============================] - 1271s 3s/step - loss: 0.0134 - val_loss: 0.1227
Epoch 6/10
499/499 [==============================] - 1362s 3s/step - loss: 0.0042 - val_loss: 0.1143
Epoch 7/10
499/499 [==============================] - 1277s 3s/step - loss: 0.0016 - val_loss: 0.1115
Epoch 8/10
499/499 [==============================] - 1297s 3s/step - loss: 0.0067 - val_loss: 0.1239
Epoch 9/10
499/499 [==============================] - 1279s 3s/step - loss: 0.0042 - val_loss: 0.1126
Epoch 10/10
499/499 [==============================] - 1378s 3s/step - loss: 8.243

---

## 5. 모델이 가사를 잘 생성하는지 확인하기

### 5-1. Model 5 결과

In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [15]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love ma little nasty girl <end> '

In [16]:
test_word = ['i love', 'i hate', 'I','you love' ,'you', 'your love', 'mine', 'love is', 'coffee', 'sweet','i am','never']
result_dict2 = {}
for word in test_word:
    result2 = generate_text(model, tokenizer, init_sentence= ' '.join(["<start>", word]))
    result_dict2[word] = result2
    print(result2)

<start> i love ma little nasty girl <end> 
<start> i hate the way you fell apart girl , its sad to see <end> 
<start> i m the hunter <end> 
<start> you love me for me could you be more phony <end> 
<start> you know i m bad , i m bad shameone , you know <end> 
<start> your love divine , oh its too much baby <end> 
<start> mine by the way i m feeling the way you can <end> 
<start> love is a woman <end> 
<start> coffee and tea <end> 
<start> sweet and undefeated <end> 
<start> i am not throwing away my shot <end> 
<start> never had money so i felt like a virgin <end> 


### 5-2 Model 6 결과

In [27]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [28]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love <end> '

In [31]:
# Model 6 결과
test_word = ['i love', 'i hate', 'I','you love' ,'you', 'your love', 'mine', 'love is', 'coffee', 'sweet','i am','never']
result_dict1 = {}
for word in test_word:
    result1 = generate_text(model, tokenizer, init_sentence= ' '.join(["<start>", word]))
    result_dict1[word] = result1
    print(result1)

<start> i love <end> 
<start> i hate <end> 
<start> i <end> 
<start> you love <end> 
<start> you <end> 
<start> your love <end> 
<start> mine <end> 
<start> love is <end> 
<start> coffee <end> 
<start> sweet <end> 
<start> i am <end> 
<start> never <end> 


### DataFrame으로 결과 확인해보기
pandas의 dataframe을 사용하여 첫 단어로 준 문장 및 단어들과 그에 따른 결과를 정리해서 확인해보았다.

loss 값은 확연하게 model 6가 낮지만, 오버 피팅 문제가 있는 것을 볼 수 있다.

Model 5 와 Model 6의 결과를 보면 Model 5가 훨씬 작사를 자연스럽게 하는 것을 볼 수 있다.

아래 결과를 보면 Model 6는 문장을 거의 만들지 못 하는 것을 볼 수 있다.

model 6는 val_loss 값만 낮고 성능은 좋지 못하다고 할 수 있을 것 같다.

In [18]:
# Model 5 결과
import pandas as pd

pd.DataFrame(list(result_dict2.items()),
                   columns=['start', 'result'])

,start,result
0,i love,<start> i love ma little nasty girl <end>
1,i hate,"<start> i hate the way you fell apart girl , i..."
2,I,<start> i m the hunter <end>
3,you love,<start> you love me for me could you be more p...
4,you,"<start> you know i m bad , i m bad shameone , ..."
5,your love,"<start> your love divine , oh its too much bab..."
6,mine,<start> mine by the way i m feeling the way yo...
7,love is,<start> love is a woman <end>
8,coffee,<start> coffee and tea <end>
9,sweet,<start> sweet and undefeated <end>


In [37]:
# Model 6 결과
import pandas as pd

pd.DataFrame(list(result_dict1.items()),
                   columns=['start', 'result'])

,start,result
0,i love,<start> i love <end>
1,i hate,<start> i hate <end>
2,I,<start> i <end>
3,you love,<start> you love <end>
4,you,<start> you <end>
5,your love,<start> your love <end>
6,mine,<start> mine <end>
7,love is,<start> love is <end>
8,coffee,<start> coffee <end>
9,sweet,<start> sweet <end>


---

## 6. 회고 
고난의 연속이었던 작사가 만들기 프로젝트.. 휴일 내내 모델만 돌린 것 같다.
덕분에 내가 많이 부족하다는 것을 다시 한번 느끼게 되었다.

사실 이전에는 loss 값도 중요하게 생각하긴 했지만 accuracy에 좀 더 집중 했던 것 같다.
이렇게 loss값에 집중한 적이 있었던가..? 하고 생각을 하다보니, 그 동안은 많은 것을 간과하고 있었던 것 같다고 느꼈다.

이번 기회를 통해 loss 값에 대해서도 많이 생각하게 되고 rnn에 대해서도 많이 배운 것 같다. 

val_loss가 너무 안잡혀서 여러번의 시도 끝에 양방향 RNN을 선택해봤다.

단순히 val_loss를 낮추기 위해 양방향 RNN을 찾아보게 되었지만, 눈에 띄게 loss가 줄어드는 것을 보니 조금 더 공부하고 싶은 마음이 들었다.

Rnn 이외에도 callback 함수, dropout 등에 대해서도 개념을 확실하게 잡질 못한 것 같다. 아직도 많이 부족하다.. 

프로젝트 제출 후에 좀 더 찾아봐야겠다.

특히 val_loss 값은 2.2 보다 정말 낮게 나왔음에도 불구하고 작사를 못하는 것에 대해서 결과는 확인했지만, 정확한 이유를 모르겠다.

이 부분 역시 다시 공부해봐야겠다.

그리고 loss 값의 변화를 시각화 하고 싶다는 생각이 들어 서치를 하다보니 keras 에서 모델 훈련 history를 확인할 수 있다는 사실도 알게되었다.

시각화 하려고 코드까지 다 짰으나 커널이 죽어버려서 히스토리가 다 날라가버렸다.

남은 데이터는 model 5, 6 뿐이었는데 그 마저도 6이 중간에 날라갔다 ㅎㅎ 

시각화의 꿈은 저 너머로 ..ㅎㅎㅠㅠ

여러모로 나를 많이 채울 수 있는 프로젝트 였다고 생각한다.

이후에 양방향 RNN을 사용을 배제하고 확실하게 loss를 잡을 수 있는 방법을 더 찾아봐야겠다.


---

## 참고 자료

[Keras Documentation](https://keras.io/ko/visualization/)

[Tensorflow 콜백함수: ReduceLROnPlateau](https://deep-deep-deep.tistory.com/56)
